# Tugas Pendahuluan
Tugas Pendahuluan dikerjakan dengan dataset titanic yang dapat didownload pada link [berikut](https://drive.google.com/file/d/16j_9FEHLjh_Y_3CdUtp9M13VwImyT89T/view?usp=sharing). Lakukan prediksi apakah suatu penumpang selamat atau tidak (kolom **survived**), bernilai 0 jika tidak selamat, dan 1 jika selamat.

<br>
Tugas dikerjakan secara berkelompok, dengan 1 kelompok terdiri atas 2 mahasiswa. Waktu pengerjaan dari 28 Maret 2022 - 3 April 2022 pukul 23.59.

# 0. Loading Data and Library

In [157]:
# Put your library here

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype

In [5]:
# Read data here

# Load titanic dataset
df_raw = pd.read_csv("titanic_dataset.csv")
df = df_raw.copy()

In [18]:
df.iloc[5]

index                                   5
pclass                                1.0
survived                              1.0
name        Chambers, Mr. Norman Campbell
sex                                  male
age                                  27.0
sibsp                                 1.0
parch                                 0.0
ticket                             113806
fare                                 53.1
cabin                                  E8
embarked                                S
Name: 5, dtype: object

# I. Data Understanding
Tujuan dari bagian ini adalah peserta dapat memahami kualitas dari data yang diberikan. Hal ini meliputi:
1. Ukuran data
2. Statistik dari tiap fitur
3. Pencilan (outlier)
4. Korelasi
5. Distribusi 

## I.1 
Carilah:
1. Ukuran dari data (instances dan features)
2. Tipe dari tiap-tiap fitur 
3. Banyaknya unique values dari fitur yang bertipe kategorikal
4. Nilai minimum, maksimum, rata-rata, median, dan standar deviasi dari fitur yang tidak bertipe kategorikal

In [126]:
# I.1 Put your code here

description = df.describe()
description

,index,pclass,survived,age,sibsp,parch,fare
count,1309.000000,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,654.000000,2.294882,0.381971,29.881135,0.498854,0.385027,33.295479
std,378.020061,0.837836,0.486055,14.413500,1.041658,0.865560,51.758668
min,0.000000,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000
25%,327.000000,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800
50%,654.000000,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200
75%,981.000000,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000
max,1308.000000,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200


## I.2
Carilah:
1. Missing values dari tiap fitur
2. Outliers dari tiap fitur (gunakan metode yang kalian ketahui)

In [145]:
# I.2 Put your code here

print("1. Missing values dari tiap fitur\n")

for column in df.columns[1:]:
    if (column != "survived"):
        print("• "+column+str(": ")+str(len(df[df[column].isna()])))

print("\n", df[df.isna().any(axis=1)])

print("\n\n2. Outliers dari tiap fitur")

for column in df.columns[1:]:
    if (is_numeric_dtype(df[column]) and column != "survived"):
        print("\n• "+column+str(":"))
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3-q1
        lower_bound = q1-1.5*iqr
        upper_bound = q3+1.5*iqr
        outliers = df.loc[(df[column]<lower_bound) | (df[column]>upper_bound)]
        print("\t- Lower bound: "+str(lower_bound))
        print("\t- Upper bound: "+str(upper_bound)+"\n")
        if (len(outliers) == 0):
            print("\t  No outliers")
        else:
            print(str(outliers))
            print("Jumlah: "+str(len(outliers)))

1. Missing values dari tiap fitur

• pclass: 0
• name: 0
• sex: 0
• age: 263
• sibsp: 0
• parch: 0
• ticket: 0
• fare: 1
• cabin: 1014
• embarked: 2

       index  pclass  survived                                          name  \
0         0     3.0       1.0                   Abelseth, Miss. Karen Marie   
1         1     3.0       0.0                       Burns, Miss. Mary Delia   
3         3     3.0       1.0  de Messemaeker, Mrs. Guillaume Joseph (Emma)   
4         4     3.0       0.0                     Jonsson, Mr. Nils Hilding   
6         6     2.0       0.0                           Enander, Mr. Ingvar   
...     ...     ...       ...                                           ...   
1301   1301     3.0       0.0              Gronnestad, Mr. Daniel Danielsen   
1302   1302     3.0       1.0                     Madsen, Mr. Fridtjof Arne   
1304   1304     3.0       1.0                         Dahl, Mr. Karl Edwart   
1307   1307     3.0       1.0                Murphy, Miss. 

Age yang NaN dapat diisi dengan age rata-rata. Fare yang NaN dapat diisi dengan fare rata-rata untuk kelas tiket bersangkutan (kelas tiket ditandai oleh fitur pclass). Embarked yang kosong dapat diisi dengan port frekuensi tertinggi pada fitur embarked.

Age yang outliers masih berada dalam rentang umur wajar bagi manusia, maka tidak akan dianggap outlier, karena data kemungkinan besar betul. SibSp yang outliers masih berada dalam rentang wajar. Pada kasus ekstrim SibSp bernilai 8 dapat mengindikasikan 8 saudara, atau 7 saudara dan 1 suami atau istri. Sebagai contoh adalah Stella Anne Sage (https://www.encyclopedia-titanica.org/titanic-victim/stella-anne-sage.html). ParCh yang outliers juga masih berada dalam rentang wajar. Data dianggap outlier karena mayoritas nilai ParCh adalah 0. Fare yang outliers dapat diteliti ulang berdasarkan sumber berikut (https://lisbdnet.com/how-much-did-tickets-cost-for-the-titanic/). Melihat nilai maksimum fare dalam data adalah £512 dan harga tiket first class diestimasi £870, serta tidak ada nilai fare yang negatif, maka data fare dapat dianggap benar. Untuk menangani dampak nilai ekstrim pada data kontinu fare, dapat dilakukan scaling/normalisasi.

## I.3
Carilah:
1. Korelasi antar fitur
2. Visualisasikan distribusi dari tiap fitur (kategorikal dan kontinu)
3. Visualisasikan distribusi dari tiap fitur, dengan data dibagi tiap unique values fitur survived

In [ ]:
# I.3 Put your code here

## I.4
Lakukanlah analisa pada data lebih lanjut jika dibutuhkan, kemudian lakukanlah:
1. Penambahan fitur jika memungkinkan
2. Pembuangan fitur yang menurut kalian tidak dibutuhkan
3. Penanganan missing values
4. Transformasi data kategorikal menjadi numerikal (encoding), dengan metode yang kalian inginkan
5. Lakukan scaling dengan MinMaxScaler

In [182]:
# I.4 Put your code here

data = {}

# Karena sebutan honorifik menandai status, maka dapat dijadikan fitur
title_count = {}

use_columns = ['pclass', 'title', 'last_name', 'sex', 'age', 'sibsp', 'parch', 'ticket_code', 'fare', 'embarked', 'survived']

for col in use_columns:
    data[col] = []
    
# investigasi fitur tambahan
for value in df.iloc:
    name = value['name']
    ticket = value['ticket']
    
    # memproses title dan last name
    commaIdx = name.find(", ")
    dotIdx = name.find(". ", commaIdx)
    title = name[commaIdx+2:dotIdx]
    last_name = name[:commaIdx]
    if (title not in title_count):
        title_count[title] = 1
    else:
        title_count[title] += 1
    
    # memproses tiket
    ticket_sep = ticket[::-1].find(" ")
    if (ticket_sep != -1):
        ticket_sep = len(ticket)-ticket_sep
        ticket_code = ticket[:ticket_sep].replace(".", "").replace(" ", "")
    else:
        if (ticket == "LINE"):
            ticket_code = "LINE"
        else:
            ticket_code = "NONE"
    
    data['pclass'].append(value['pclass'])
    data['title'].append(title)
    data['last_name'].append(last_name)
    data['sex'].append(value['sex'])
    data['age'].append(value['age'])
    data['sibsp'].append(value['sibsp'])
    data['parch'].append(value['parch'])
    data['ticket_code'].append(ticket_code)
    data['fare'].append(value['fare'])
    data['embarked'].append(value['embarked'])
    data['survived'].append(value['survived'])

# Data understanding kolom title count
print("Title count:")
for title in title_count:
    print("• "+title+": "+str(title_count[title]))

# Untuk memudahkan, beberapa sebutan digabung, terutama yang frekuensiya jarang dalam dataset
# selebihnya, sebutan wanita dapat digabung menjadi Mrs dan Miss, sesuai
# https://newrepublic.com/article/119432/history-female-titles-mistress-miss-mrs-or-ms
titles_map = {'Miss': 'Miss', 'Mrs': 'Mrs', 'Mr': 'Mr', 'Master': 'Master', 'Dr': 'Other', 'Rev': 'Other', 'Don': 'Other', 'Col': 'Army', 'Mme': 'Mrs', 'Capt': 'Army', 'Sir': 'Other', 'Ms': 'Mrs', 'the Countess': 'Nobility', 'Dona': 'Other', 'Mlle': 'Miss', 'Major': 'Army', 'Lady': 'Nobility', 'Jonkheer': 'Other'}

for i, title in enumerate(data['title']):
    data['title'][i] = titles_map[title]

print("\nDataset dengan fitur ditambahkan/dihilangkan:")
new_df = pd.DataFrame(data)
print(new_df)

# Penanganan missing values
new_df.describe()

new_df['age'] = new_df['age'].fillna(new_df['age'].mean())
# for each passanger class, fill missing fare as average fare of each passanger class
for pclass in new_df['pclass'].unique():
    avg_fare_of_pclass = new_df[(new_df['pclass'] == pclass)][['fare']].mean()[0]
    new_df[(new_df['pclass'] == pclass)][['fare']] = new_df[(new_df['pclass'] == pclass)][['fare']].fillna(avg_fare_of_pclass)

new_df['embarked'] = new_df['embarked'].fillna(new_df['embarked'].mode())

Title count:
• Miss: 260
• Mrs: 197
• Mr: 757
• Master: 61
• Dr: 8
• Rev: 8
• Don: 1
• Col: 4
• Mme: 1
• Capt: 1
• Sir: 1
• Ms: 2
• the Countess: 1
• Dona: 1
• Mlle: 2
• Major: 2
• Lady: 1
• Jonkheer: 1

Dataset dengan fitur ditambahkan/dihilangkan:
      pclass title             last_name     sex   age  sibsp  parch  \
0        3.0  Miss              Abelseth  female  16.0    0.0    0.0   
1        3.0  Miss                 Burns  female  18.0    0.0    0.0   
2        1.0  Miss               Fortune  female  24.0    3.0    2.0   
3        3.0   Mrs        de Messemaeker  female  36.0    1.0    0.0   
4        3.0    Mr               Jonsson    male  27.0    0.0    0.0   
...      ...   ...                   ...     ...   ...    ...    ...   
1304     3.0    Mr                  Dahl    male  45.0    0.0    0.0   
1305     1.0    Mr  Penasco y Castellana    male  18.0    1.0    0.0   
1306     2.0  Miss                Becker  female  12.0    2.0    1.0   
1307     3.0  Miss            

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [183]:
for column in new_df.columns[1:]:
    if (column != "survived"):
        print("• "+column+str(": ")+str(len(new_df[new_df[column].isna()])))

print("\n", new_df[new_df.isna().any(axis=1)])

• title: 0
• last_name: 0
• sex: 0
• age: 0
• sibsp: 0
• parch: 0
• ticket_code: 0
• fare: 1
• embarked: 2

      pclass title last_name     sex   age  sibsp  parch ticket_code  fare  \
268     1.0  Miss     Icard  female  38.0    0.0    0.0        NONE  80.0   
387     1.0   Mrs     Stone  female  62.0    0.0    0.0        NONE  80.0   
911     3.0    Mr    Storey    male  60.5    0.0    0.0        NONE   NaN   

    embarked  survived  
268      NaN       1.0  
387      NaN       1.0  
911        S       0.0  


# II. Experiments Design
Tujuan dari bagian ini adalah peserta dapat memahami cara melakukan eksperimen mencari metode terbaik dengan benar. Hal ini meliputi:
1. Pembuatan model
2. Proses validasi
3. Hyperparameter tuning

## II.1
Tentukanlah metrics yang akan digunakan pada eksperimen kali ini (dapat lebih dari 1 metric)

Put your answer for section II.1 here

## II.2 
Bagi data dengan perbandingan 0.8 untuk data train dan 0.2 untuk data validasi

In [ ]:
# II.2 Put your code here

## II.3
Lakukanlah:
1. Prediksi dengan menggunakan model Logistic Regression sebagai *baseline*
2. Tampilkan evaluasi dari model yang dibangun dari metrics yang anda tentukan pada II.1
3. Tampilkan confusion matrix

In [ ]:
# II.3 Put your code here

## II.4 
Lakukanlah:
1. Pembelajaran dengan model lain
2. Hyperparameter tuning model yang kalian pakai dengan menggunakan Grid Search (perhatikan random factor pada beberapa algoritma model)
3. Lakukan validasi dengan menggunakan cross validation


In [ ]:
# II.4 Put your code here

# III. Improvement
Terdapat beberapa metode untuk melakukan peningkatan performa, contohnya adalah:
1. Melakukan oversampling / undersampling pada data
2. Menggabungkan beberapa model 

Pada bagian ini, kalian diharapkan dapat:
1. Melakukan training dengan data hasil oversampling / undersampling dan melakukan validasi dengan benar
2. Memahami beberapa metode untuk menggabungkan beberapa model

## III.1
Lakukanlah:
1. Oversampling pada kelas minoritas pada data train, kemudian train dengan model *baseline* (II.3), lakukan validasi dengan data validasi. Data train dan validasi adalah data yang kalian bagi pada bagian II.2
2. Undersampling pada kelas mayoritas pada data train, kemudian train dengan model *baseline* (II.3) lakukan validasi dengan data validasi. Data train dan validasi adalah data yang kalian bagi pada bagian II.2

In [ ]:
# III.1 Put your code here

## III.2
Lakukanlah:
1. Eksplorasi soft voting, hard voting, dan stacking
2. Buatlah model Logistic Regression dan SVM (boleh menggunakan model dengan beberapa parameter yang berbeda)
3. Lakukanlah soft voting dari model-model yang sudah kalian buat pada poin 2
4. Lakukan hard voting dari model-model yang sudah kalian buat pada poin 2
5. Lakukanlah stacking dengan final classifier adalah Logistic Regression dari model-model yang sudah kalian buat pada poin 2
6. Lakukan validasi dengan metrics yang kalian tentukan untuk poin 3, 4, dan 5

Put your answer for section III.2 point 1 here

In [ ]:
# III.2 Put your code here

# IV. Analisis
Bandingkan hasil dari:
1. Model Baseline (II.3)
2. Model lain (II.4)
3. Hasil undersampling
4. Hasil oversampling
5. Hasil soft voting
6. Hasil hard voting
7. Hasil stacking 

Put your answer for section IV here